In [1]:
if !@isdefined(PACKAGES)
    using Pkg;
    Pkg.add("Arpack");
    using Arpack;
    PACKAGES = true
end

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


true

We will encode a chain in the Ising model as a binary number.

0 - spin down
1 - spin up

For example in an 8-spin chain we would like to know the value of the 3-rd spin (counting right to left/least significant to most significant bit, starting from 0):

```
10100110
    ^
    |
    ------ 3-rd most significant bit
```
This can be achieved via bit-mask testing.

```
Let
mask = 00000001
spin_no = 3

mask = mask << spin_no
mask = 00001000

Using the bitwise-and operation then:
  10100110
& 
  00001000
------------
  00000000 == -> spin down
```

If the results is zero the bit was unset (spin down), otherwise it was set (spin up).

Putting into practice:

In [2]:
# Example
state = 0b10100110
spin_num = 3
mask = 1 << spin_num
println(bitstring((state)))
println("&")
println(bitstring(Int8(mask)))
println("--------")
ss = state & mask
println(bitstring(Int8(ss)))
ss != 0 ? "up" : "down"

10100110
&


00001000
--------
00000000


"down"

To make counting easier later, we shall offset all the bits by 1, i.e. the least significant bit having an index of 1 instead of 0

In [3]:
function spin_state(chain_configuration, spin_number)
    spin_number -= 1 # start counting of bits from 1
    mask = 1 << spin_number
    state = chain_configuration & mask
    return state != 0 ? 1 : -1
end

spin_state (generic function with 1 method)

In [4]:
function R(i, p)
    return (1.0/i)^p
end

R (generic function with 1 method)

This implementation should be much faster than using loops since bit-shifts are low-level single instruction operations

Now, we can define a function that takes two binary encoded neighbouring chains and calculates the hamiltonian:

$$
    H = -J \sum_{ij} s_i \cdot s_j - h \sum_{i} s_i
$$

In [5]:
function hamiltonian(
                     chain_size, # number of spins in chain
                     interaction_range, # range of interactions in chain
                     p, # interaction exponent
                     right_chain, # encoded i-th chain
                     left_chain, # encoded i+1-th chain
                     Rs, # side-to-side interaction strength,
                     M, # external magnetic field
                     )
    
    h = 0.0 # start with 0 hamiltonian
    for spin_num in 1:chain_size # calculate neighbouring row (side-to-side) interactions
        #println(spin_state(right_chain, spin_num))
        h += spin_state(right_chain, spin_num)*spin_state(left_chain, spin_num)
    end
    h *= Rs

    for inter_offset in 1:interaction_range # calculate in-chain NN interactions for both chains
        h_temp = 0.0 # hamiltonian for given interaction offset
        for spin_num in 1:chain_size
            nbr_num = spin_num + inter_offset # index of iteracting neighbour
            if nbr_num > chain_size
                nbr_num = nbr_num % chain_size
            end
            h_temp += spin_state(right_chain, spin_num)*spin_state(right_chain, nbr_num) + spin_state(left_chain, spin_num)*spin_state(left_chain, nbr_num)
        end
        h += 0.5 * R(inter_offset, p) * h_temp # devide by 2 because of double-counting of chains
    end
    
    h_temp = 0.0
    for spin_num in 1:chain_size # external field interaction
        h_temp += spin_state(right_chain, spin_num) + spin_state(left_chain, spin_num)
    end

    h += 0.5 * M * h_temp
    
    return h
end

hamiltonian (generic function with 1 method)

In [6]:
function generate_transfer_matrix(chain_size, interaction_range, p, Rs, M, temperature)
    n  = 2^chain_size
    t_mtx = zeros((n, n))
    for i in 1:n
        for j in 1:n
            h_n = hamiltonian(chain_size, interaction_range, p, i, j, Rs, M)
            t_mtx[i, j] = exp(h_n/temperature)
        end
    end
    return t_mtx
end

generate_transfer_matrix (generic function with 1 method)

In [7]:
ch_size = 8
int_range = 1
p = 2
Rs = 1
M = 0
T = 2
transfer_mtx = generate_transfer_matrix(ch_size, int_range, p, Rs, M, T);

In [8]:
eig_val, eig_vec = eigs(transfer_mtx, nev=10);
eig_val

10-element Vector{Float64}:
  3709.959215993373
 -3613.8396165338017
  1513.108055242081
 -1320.2010755163271
 -1320.2010755163246
   803.8016131810996
   803.8016131810986
   803.8016131810986
   803.8016131810975
  -761.0346661450451